In [1]:
from __future__ import print_function, absolute_import

import os
import sys
import time
import copy

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

import cameras
import data_utils
import linear_model
import procrustes
import viz
import glob
import cdflib
sys.argv = sys.argv[:1]

import cv2


import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import data_process as data_process

import json 
from model import LinearModel, weight_init
import torch.nn as nn
import utils as utils


sys.path.append(os.path.join(os.path.dirname('__file__'), "progress"))

from progress.bar import Bar as Bar

# Load Human3.6M Skeleton

import torch.nn as nn
import torch.optim
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
from torch.autograd import Variable

import pickle

tf.app.flags.DEFINE_string("action","All", "The action to train on. 'All' means all the actions")

# Directories
tf.app.flags.DEFINE_string("cameras_path","../data/h36m/metadata.xml", "File with h36m metadata, including cameras")

FLAGS = tf.app.flags.FLAGS


# Initiate Function

In [2]:
SUBJECT_IDS = [1,5,6,7,8,9,11]
this_file = os.path.dirname(os.path.realpath('__file__'))

#Load metadata.xml camera
rcams = cameras.load_cameras(os.path.join(this_file, FLAGS.cameras_path), SUBJECT_IDS)

In [3]:



# Joints in H3.6M -- data has 32 joints, but only 17 that move; these are the indices.
H36M_NAMES = ['']*32
H36M_NAMES[0]  = 'Hip'
H36M_NAMES[1]  = 'RHip'
H36M_NAMES[2]  = 'RKnee'
H36M_NAMES[3]  = 'RFoot'
H36M_NAMES[6]  = 'LHip'
H36M_NAMES[7]  = 'LKnee'
H36M_NAMES[8]  = 'LFoot'
H36M_NAMES[12] = 'Spine'
H36M_NAMES[13] = 'Thorax'
H36M_NAMES[14] = 'Neck/Nose'
H36M_NAMES[15] = 'Head'
H36M_NAMES[17] = 'LShoulder'
H36M_NAMES[18] = 'LElbow'
H36M_NAMES[19] = 'LWrist'
H36M_NAMES[25] = 'RShoulder'
H36M_NAMES[26] = 'RElbow'
H36M_NAMES[27] = 'RWrist'

index_alphapose={
    # Use 17 skeleton point
    "Nose": 0,
    "RShoulder": 6,
    "RElbow": 8,
    "RWrist": 10,
    "LShoulder": 5,
    "LElbow": 7,
    "LWrist": 9,
    "RHip": 12,
    "RKnee": 14,
    "RAnkle": 16,
    "LHip": 11,
    "LKnee": 13,
    "LAnkle": 15,
    "REye": 2,
    "LEye": 1,
    "REar": 4,
    "LEar": 3
}




index_mapping={
# Alpha Pose to Human 3.6M
"Hip": [20, 0],
"RHip": [12,1],
"RKnee": [14,2],
"RFoot": [16,3],
"LHip": [11,6],
"LKnee": [13,7],
"LFoot": [15,8],
"Spine": [19,12],
"Thorax": [18,13],
"Head": [17,15],
"LShoulder": [5,17],
"LElbow": [7,18],
"LWrist": [9,19],
"RShoulder": [6,25],
"RElbow": [8,26],
"RWrist": [10,27]
}


def data_converter(data):
    data=data['keypoints']
    keypoints=[]
    kp_score=[]
    for a in range (0,len(data)):
        score=[]
        if ((a+3)%3==0):
            keypoints.append(data[a])
            keypoints.append(data[a+1])
        elif((a+1)%3==0):
            score=data[a]
            kp_score.append(score)

    return keypoints
    




In [4]:


def count_head(alpha_pose):
    x = (alpha_pose[index_alphapose['LEar']*2]+alpha_pose[index_alphapose['REar']*2])/2
    y = (alpha_pose[index_alphapose['LEar']*2+1]+alpha_pose[index_alphapose['REar']*2+1])/2
    return x,y


def count_thorax(alpha_pose):
    x = (alpha_pose[index_alphapose['LShoulder']*2]+alpha_pose[index_alphapose['RShoulder']*2])/2
    y = (alpha_pose[index_alphapose['LShoulder']*2+1]+alpha_pose[index_alphapose['RShoulder']*2+1])/2
    return x,y


def count_spine(alpha_pose):
    hip_x,hip_y=count_hip(alpha_pose)
    thorax_x,thorax_y=count_thorax(alpha_pose)
    x = (hip_x+thorax_x)/2
    y = (hip_y+thorax_y)/2
    return x,y

def count_hip(alpha_pose):
    x = (alpha_pose[index_alphapose['LHip']*2]+alpha_pose[index_alphapose['RHip']*2])/2
    y = (alpha_pose[index_alphapose['LHip']*2+1]+alpha_pose[index_alphapose['RHip']*2+1])/2
    return x,y

def add_features(alpha_pose):
    #Count Head
    head_x,head_y=count_head(alpha_pose)
    alpha_pose=np.append(alpha_pose,(head_x,head_y))
    
    #Count Thorax
    thorax_x,thorax_y=count_thorax(alpha_pose)
    alpha_pose=np.append(alpha_pose,(thorax_x,thorax_y))
 
    
    #Count Spine
    spine_x,spine_y=count_spine(alpha_pose)
    alpha_pose=np.append(alpha_pose,(spine_x,spine_y))
    
    #Count Hip
    hip_x,hip_y=count_hip(alpha_pose)
    alpha_pose=np.append(alpha_pose,(hip_x,hip_y))
    
    return alpha_pose




In [5]:
def map_alpha_to_human(alpha_pose):
    alpha_pose=add_features(alpha_pose)
    temp_list = [None] * 64
    for a,b in index_mapping.items():
        temp_list[b[1]*2]=alpha_pose[b[0]*2]
        temp_list[b[1]*2+1]=alpha_pose[b[0]*2+1]
    human36m=np.asarray(temp_list)
    return human36m

# Do 3D Prediction from Custom Video

## Using Created Statistic Dictionary

In [6]:
normalize=True
actions = data_utils.define_actions( FLAGS.action )
# Human3.6m IDs for training and testing
TRAIN_SUBJECTS = [1,5,6,7,8]
TEST_SUBJECTS  = [9,11]


stat_3D = torch.load('../data/stat_3d.pth.tar')
stat_2D = torch.load('../data/stat_2d.pth.tar')




In [7]:

data_mean_2d, data_std_2d, dim_to_ignore_2d, dim_to_use_2d = stat_2D['mean'],stat_2D['std'],stat_2D['dim_ignore'],stat_2D['dim_use']
data_mean_3d, data_std_3d, dim_to_ignore_3d, dim_to_use_3d = stat_3D['mean'],stat_3D['std'],stat_3D['dim_ignore'],stat_3D['dim_use']

In [8]:
#All the json data
#load json data using json,load(f)
#!/usr/bin/env python
# -*- coding: utf-8 -*-


class Human36M_testing(Dataset):
    def __init__(self, skeleton,many=False):
        """
        :param actions: list of actions to use
        :param data_path: path to dataset
        :param use_hg: use stacked hourglass detections
        :param is_train: load train/test dataset
        """

        self.test_inp, self.test_out = [], []

        # loading data
        # load test data
       
        if many:
            num_f= skeleton.shape
            for i in range(num_f[0]):
                self.test_inp.append(skeleton[i])
        else:
            self.test_inp.append(skeleton)


    def __getitem__(self, index):
        inputs = torch.from_numpy(self.test_inp[index]).float()

        return inputs

    def __len__(self):
        return len(self.test_inp)
    
def normalize_single_data(data, data_mean, data_std, dim_to_use ):
    """Normalizes a dictionary of poses

    Args
    data: dictionary where values are
    data_mean: np vector with the mean of the data
    data_std: np vector with the standard deviation of the data
    dim_to_use: list of dimensions to keep in the data
    Returns
    data_out: dictionary with same keys as data, but values have been normalized
    """

    data= data[dim_to_use]
    mu = data_mean[dim_to_use]
    stddev = data_std[dim_to_use]
    data_out= np.divide( (data - mu), stddev )

    return data_out
def create_datatest(data):
    converted=[]
    for dat in data:
        convert=np.asarray(data_converter(dat))
        human36m_alpha_example=map_alpha_to_human(convert)
        normalized=normalize_single_data(human36m_alpha_example,data_mean_2d,data_std_2d,dim_to_use_2d)
        normalized=normalized.astype('float')
        converted.append(normalized)

    converted=np.asarray(converted) 
    test_loader = DataLoader(
        dataset=Human36M_testing(converted,True),
        batch_size=1024,
        shuffle=False,
        num_workers=0,
        pin_memory=True)
    return test_loader

# Load Model

In [9]:


model_path='../../3d_pose_baseline_pytorch/checkpoint/test/ckpt_best.pth.tar'
# create model
print(">>> creating model")
model = LinearModel()
model = model.cuda()
model.apply(weight_init)
print(">>> total params: {:.2f}M".format(sum(p.numel() for p in model.parameters()) / 1000000.0))
criterion = nn.MSELoss(size_average=True).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1.0e-3)

print(">>> loading ckpt from '{}'".format('../../3d_pose_baseline_pytorch/checkpoint/test/ckpt_best.pth.tar'))
ckpt = torch.load(model_path)
start_epoch = ckpt['epoch']
err_best = ckpt['err']
glob_step = ckpt['step']
lr_now = ckpt['lr']
model.load_state_dict(ckpt['state_dict'])
optimizer.load_state_dict(ckpt['optimizer'])
print(">>> ckpt loaded (epoch: {} | err: {})".format(start_epoch, err_best))

new_stat_3d={}
new_stat_3d['mean']=data_mean_3d
new_stat_3d['std']=data_std_3d
new_stat_3d['dim_use']=dim_to_use_3d
new_stat_3d['dim_ignore']=dim_to_ignore_3d
    
def test(test_loader, model, criterion, stat_3d, procrustes=False):
    losses = utils.AverageMeter()
    model.eval()

    all_dist = []
    pred_result=[]
    start = time.time()
    batch_time = 0
    bar = Bar('>>>', fill='>', max=len(test_loader))

    for i, inps in enumerate(test_loader):
        inputs = Variable(inps.cuda())
        
        with torch.no_grad():
            outputs = model(inputs)

        # calculate erruracy
        print(outputs.shape)
        outputs_unnorm = data_process.unNormalizeData(outputs.data.cpu().numpy(), stat_3d['mean'], stat_3d['std'], stat_3d['dim_use'])

        # remove dim ignored
        dim_use = np.hstack((np.arange(3), stat_3d['dim_use']))

        outputs_use = outputs_unnorm[:, dim_use]
        pred_result.append(outputs_unnorm)
        
        # update summary
        if (i + 1) % 100 == 0:
            batch_time = time.time() - start
            start = time.time()

        bar.suffix = '({batch}/{size}) | batch: {batchtime:.4}ms | Total: {ttl} | ETA: {eta:} | loss: {loss:.6f}' \
            .format(batch=i + 1,
                    size=len(test_loader),
                    batchtime=batch_time * 10.0,
                    ttl=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg)
        bar.next()
    bar.finish()
    return pred_result

# Combine prediction from each batch into one prediction
def combine_prediction(pred_result_all):
    prediction_list = []
    for pred in pred_result_all:
        for pre in pred:
            prediction_list.append(pre)
    prediction_list=np.asarray(prediction_list)
    return prediction_list


def correct_3D(poses3d_input,poses2d_normalized):
    _max = 0
    _min = 10000
    poses3d=np.copy(poses3d_input)
    
    spine_x = poses2d_normalized[0][24]
    spine_y = poses2d_normalized[0][25]
            
    
    for i in range(poses3d.shape[0]):

        for j in range(32):

            tmp = poses3d[i][j * 3 + 2]
            poses3d[i][j * 3 + 2] = poses3d[i][j * 3 + 1]
            poses3d[i][j * 3 + 1] = tmp
            if poses3d[i][j * 3 + 2] > _max:
                _max = poses3d[i][j * 3 + 2]
                print("_max: ",_max)
            if poses3d[i][j * 3 + 2] < _min:
                _min = poses3d[i][j * 3 + 2]
                print("_min: ",_min)

    for i in range(poses3d.shape[0]):
        for j in range(32):
            poses3d[i][j * 3 + 2] = _max - poses3d[i][j * 3 + 2] + _min
            poses3d[i][j * 3] += (spine_x - 630)
            poses3d[i][j * 3 + 2] += (500 - spine_y)

    return poses3d

def inferencealphaposeto3D(path):
    # Opening JSON file 
    f = open(path) 

    # returns JSON object as  
    # a dictionary 
    data = json.load(f) 


    # Closing file 
    f.close() 
    all_test_data=create_datatest(data)
    
    pred_result_all=test(all_test_data, model, criterion, new_stat_3d) #All
    
    prediction_list=combine_prediction(pred_result_all)
    
    test_2d_normalized = np.asarray(all_test_data.dataset.test_inp) 
    fixed=correct_3D(prediction_list,test_2d_normalized)
    
    base=os.path.basename(path)
    base=os.path.splitext(base)[0]
    
    with open('../inference_result_npy/'+base+'.npy', 'wb') as f:
        np.save(f, fixed)
    
    return fixed

>>> creating model
>>> total params: 4.29M
>>> loading ckpt from '../../3d_pose_baseline_pytorch/checkpoint/test/ckpt_best.pth.tar'
>>> ckpt loaded (epoch: 147 | err: 42.151899649376176)


D:\Project Lab\THESIS_FALL_DETECTION\2D to 3D Pose\3d_pose_baseline_pytorch\src\model.py:11: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight)
C:\Users\hutom\anaconda3\envs\3DPose\lib\site-packages\torch\nn\_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [11]:
path='../json_test/AlphaPose_FallingDown_01_Angle 2.json'
inference_result=inferencealphaposeto3D(path)

torch.Size([1024, 48])
torch.Size([1024, 48])
torch.Size([1024, 48])
torch.Size([717, 48])
_min:  0.0
_max:  23.74018353833017
_max:  380.1990142900833
_max:  714.0541282291732
_max:  742.1497253356404
_min:  -23.740053932106353
_max:  748.6368635545936
_min:  -193.1940967258103
_min:  -367.2188688863604
_min:  -435.08890570553115
_min:  -435.8228647302801
_min:  -448.7604641562079
_min:  -449.1551857555212
_min:  -449.34476256537164
_max:  749.7844913365291
_max:  750.0728797480272
_max:  750.0797652940556
_max:  752.3011221701864
_max:  753.9536894567138
_max:  754.0561101419001
_max:  754.0572770607534
_max:  756.1461777751277
_max:  756.2031031209244
_max:  756.3576655094037
_min:  -456.17790766677865
_min:  -547.0309456177768
_min:  -566.648016773134
_min:  -591.0398371447454
_min:  -623.1972695410624
_min:  -636.5362085175566
_min:  -637.8869959349664
_min:  -639.1818992973043
_min:  -640.3048061233621
_min:  -642.1744767971251
_min:  -642.8402653119958
_min:  -642.8405109990489
